In [1]:
"""
nn_entity_v1.py

Initial version by R. Mukai 4 March 2018

Updated version 24 December 2018.  This code is used for our
initial paper on two cooperating agents with knowledge states
solving a simple propositional logic problem.
"""

import pickle
import numpy as np

from keras.models import Model
from keras.layers import Input
from keras.layers import Bidirectional, TimeDistributed
from keras.layers.core import Activation, Dense, Dropout
from keras.layers.recurrent import LSTM
from keras.layers.merge import concatenate
from keras.models import load_model

from nn_utils.problem_generator import *
from nn_utils.string_and_array import *

DATA_PATH = "../data"

class NN_Entity_1:

    def __init__(self, id_number,
                 initial_sentences = [],
                 nn_file = None, data_set_file = None,
                 gen_data_sets=True, start_training_epoch=0, max_training_epoch=1024, max_data_sets=64,
                 num_instances=10000
                 ):
        self.DATA_PATH = DATA_PATH
        # Create network if nn_file is None.  Otherwise
        # just load existing network.

        if nn_file is None:

            self.generate_network(gen_data_sets, start_training_epoch, max_training_epoch, max_data_sets,
                 num_instances)

        else:

            self.model = load_model(nn_file)

            # Now decide whether to use existing data
            # if supplied or make new data set.

            if data_set_file is not None:

                fp = open('%s_X1.npy' % (data_set_file,), 'rb')
                X = np.load(fp)
                fp.close()

                fp = open('%s_Y1.npy' % (data_set_file,), 'rb')
                Y1 = np.load(fp)
                fp.close()

                fp = open('%s_Y2.npy' % (data_set_file,), 'rb')
                Y2 = np.load(fp)
                fp.close()

                fp = open('%s_one_hot_dictionary.pck' % (data_set_file,), 'rb')
                one_hot_dictionary = pickle.load(fp)
                fp.close()

                fp = open('%s_template_choices.pck' % (data_set_file,), 'rb')
                template_choices = pickle.load(fp)
                fp.close()

                fp = open('%s_question_template_list.pck' % (data_set_file,), 'rb')
                question_template_list = pickle.load(fp)
                fp.close()

                fp = open('%s_answer_template_list.pck' % (data_set_file,), 'rb')
                answer_template_list = pickle.load(fp)
                fp.close()

                self.data_set={}

                self.data_set['X'] = X
                self.data_set['Y1'] = Y1
                self.data_set['Y2'] = Y2
                self.data_set['one_hot_dictionary'] = one_hot_dictionary
                self.data_set['template_choices'] = template_choices
                self.data_set['question_template_list'] = question_template_list
                self.data_set['answer_template_list'] = answer_template_list

                (b, self.m, self.n) = self.data_set['X'].shape

            else:

                template_list = return_simple_propositional_templates()

                main_template_list = []

                num_instances = 10000

                num_templates = len(template_list)

                template_error_scorecard = {}
                for index in range(num_templates):
                    template_error_scorecard[index] = [0, 0]

                template_choices = []

                for index in range(num_instances):
                    random_index = np.random.randint(0, num_templates)
                    template_choices.append(random_index)
                    main_template_list.append(template_list[random_index])

                result_list, X, Y, one_hot_dictionary = template_list_to_problem_set(num_vars=10,
                                                                                     template_list=main_template_list)

                data_set = {}
                data_set['X'] = X
                data_set['Y'] = Y
                data_set['one_hot_dictionary'] = one_hot_dictionary
                data_set['template_choices'] = template_choices

                self.data_set = data_set

                (b, self.m, self.n) = self.data_set['X'].shape

            # End if-then logic for data set

        # End if-then logic for making new network

        # Set up an entity number for this entity.
        # When referring to itself, it should use "me".
        # When other entities refer to it, it should be
        # "n<entity_number>", for example "n7" if entity
        # number is 7.  Also, set up initial sentences this
        # entity knows, if any.

        self.entity_id_and_knowledge = {
            'id_number' : id_number,
            'id_string' : 'n'+str(id_number),
            'sentence_list' : copy.deepcopy(initial_sentences)
        }

    # End initializer

    def generate_network(self, gen_data_sets=True, start_training_epoch=0, max_training_epoch=1024, max_data_sets=64,
                         num_instances = 10000):
        template_list = return_simple_propositional_templates()

        # Next, create the list of actual question and answer templates.

        new_templates_distilled_question = []

        new_templates_answer = []

        num_template_repeats = 5

        for template in template_list:

            for index in range(template['max_instances']):
                new_pair = create_problem_with_repetition(template, max_rep=3)

                new_templates_distilled_question.append(new_pair[0])
                new_templates_answer.append(new_pair[1])

        # End loop

        num_templates = len(new_templates_distilled_question)

        template_error_scorecard = {}
        for index in range(num_templates):
            template_error_scorecard[index] = [0, 0, 0]

        template_choices = []

        if gen_data_sets:
            for data_set_index in range(max_data_sets):

                template_choices = []
                question_template_list = []
                answer_template_list = []

                for index in range(num_instances):
                    random_index = np.random.randint(0, num_templates)
                    template_choices.append(random_index)
                    question_template_list.append(new_templates_distilled_question[random_index])
                    answer_template_list.append(new_templates_answer[random_index])

                one_hot_dictionary = gen_one_hot_encoding(num_vars=10)

                result_list_q, result_list_a, X1, Y1, Y2, one_hot_dictionary = ext_template_list_to_problem_set(num_vars=10,
                                                                                                                template_list=(
                                                                                                                question_template_list,
                                                                                                                answer_template_list),
                                                                                                                one_hot_dictionary=one_hot_dictionary)

                data_set = {}
                data_set['X'] = X1
                data_set['Y1'] = Y1
                data_set['Y2'] = Y2
                data_set['one_hot_dictionary'] = one_hot_dictionary
                data_set['template_choices'] = template_choices
                data_set['question_template_list'] = question_template_list
                data_set['answer_template_list'] = answer_template_list

                fp = open('%s/data_set_%d_X1.npy' % (self.DATA_PATH,data_set_index,), 'wb')
                np.save(fp, X1)
                fp.close()

                fp = open('%s/data_set_%d_Y1.npy' % (self.DATA_PATH,data_set_index,), 'wb')
                np.save(fp, Y1)
                fp.close()

                fp = open('%s/data_set_%d_Y2.npy' % (self.DATA_PATH,data_set_index,), 'wb')
                np.save(fp, Y2)
                fp.close()

                fp = open('%s/data_set_%d_one_hot_dictionary.pck' % (self.DATA_PATH,data_set_index,), 'wb')
                pickle.dump(one_hot_dictionary, fp)
                fp.close()

                fp = open('%s/data_set_%d_template_choices.pck' % (self.DATA_PATH,data_set_index,), 'wb')
                pickle.dump(template_choices, fp)
                fp.close()

                fp = open('%s/data_set_%d_question_template_list.pck' % (self.DATA_PATH,data_set_index,), 'wb')
                pickle.dump(question_template_list, fp)
                fp.close()

                fp = open('%s/data_set_%d_answer_template_list.pck' % (self.DATA_PATH,data_set_index,), 'wb')
                pickle.dump(answer_template_list, fp)
                fp.close()

            # End data set generator loop

        # End data set generator code

        if gen_data_sets:

            (mx, nx) = X1[0].shape

        else:
            fp = open('%s/data_set_%d_X1.npy' % (self.DATA_PATH,0,), 'rb')
            X1 = np.load(fp)
            (mx, nx) = X1[0].shape
            fp.close()

            fp = open('%s/data_set_%d_Y1.npy' % (self.DATA_PATH,0,), 'rb')
            Y1 = np.load(fp)
            fp.close()

            fp = open('%s/data_set_%d_Y2.npy' % (self.DATA_PATH,0,), 'rb')
            Y2 = np.load(fp)
            fp.close()

            fp = open('%s/data_set_%d_one_hot_dictionary.pck' % (self.DATA_PATH,0,), 'rb')
            one_hot_dictionary = pickle.load(fp)
            fp.close()

            fp = open('%s/data_set_%d_template_choices.pck' % (self.DATA_PATH,0,), 'rb')
            template_choices = pickle.load(fp)
            fp.close()

            fp = open('%s/data_set_%d_question_template_list.pck' % (self.DATA_PATH,0,), 'rb')
            question_template_list = pickle.load(fp)
            fp.close()

            fp = open('%s/data_set_%d_answer_template_list.pck' % (self.DATA_PATH,0,), 'rb')
            answer_template_list = pickle.load(fp)
            fp.close()

        if start_training_epoch == 0:

            inputs = Input(shape=(None, nx))

            x1 = LSTM(256, return_sequences=True)(inputs)

            x2 = LSTM(256, return_sequences=True)(x1)

            n_y1_batch, n_y1_timesteps, n_y1_size = Y1.shape

            n_y2_batch, n_y2_timesteps, n_y2_size = Y2.shape

            x3 = (Dense(Y1[0, 0].size))(x1)

            y1 = Activation('softmax')(x3)

            x4 = (Dense(Y1[0, 0].size))(x2)

            y2 = Activation('softmax')(x4)

            model = Model(inputs=inputs, outputs=[y1, y2])

            model.compile(optimizer='Adam', loss='categorical_crossentropy',
                          metrics=['accuracy'])

            model.save("%s/untrained_dual_output.h5" % (self.DATA_PATH,))

        else:

            model = load_model("%s/trained_model_prop_new_%d.h5" % (self.DATA_PATH,start_training_epoch-1,))

        model.summary()

        for index in range(start_training_epoch, max_training_epoch):

            data_set_index = np.random.randint(0, max_data_sets)

            fp = open('%s/data_set_%d_X1.npy' % (self.DATA_PATH,data_set_index,), 'rb')
            X = np.load(fp)
            fp.close()

            fp = open('%s/data_set_%d_Y1.npy' % (self.DATA_PATH,data_set_index,), 'rb')
            Y1 = np.load(fp)
            fp.close()

            fp = open('%s/data_set_%d_Y2.npy' % (self.DATA_PATH,data_set_index,), 'rb')
            Y2 = np.load(fp)
            fp.close()

            fp = open('%s/data_set_%d_one_hot_dictionary.pck' % (self.DATA_PATH,data_set_index,), 'rb')
            one_hot_dictionary = pickle.load(fp)
            fp.close()

            fp = open('%s/data_set_%d_one_hot_dictionary.pck' % (self.DATA_PATH,data_set_index,), 'rb')
            one_hot_dictionary = pickle.load(fp)
            fp.close()

            fp = open('%s/data_set_%d_template_choices.pck' % (self.DATA_PATH,data_set_index,), 'rb')
            template_choices = pickle.load(fp)
            fp.close()

            fp = open('%s/data_set_%d_question_template_list.pck' % (self.DATA_PATH,data_set_index,), 'rb')
            question_template_list = pickle.load(fp)
            fp.close()

            fp = open('%s/data_set_%d_answer_template_list.pck' % (self.DATA_PATH,data_set_index,), 'rb')
            answer_template_list = pickle.load(fp)
            fp.close()

            model.fit(X, [Y1, Y2], epochs=1, validation_split=0.2)
            # model.fit(X, Y2, epochs=1, validation_split=0.2)

            if index % 16 == 0:

                model.save("%s/trained_model_prop_new_%d.h5" % (self.DATA_PATH,index,))

        [Y1_hat, Y2_hat] = model.predict(X)
        # Y2_hat = model.predict(X1)

        (b1, m1, n1) = Y1.shape

        (b2, m2, n2) = Y2.shape

        # Copy key data to self.

        self.model = model
        data_set = {}
        data_set['X'] = X1
        data_set['Y1'] = Y1
        data_set['Y2'] = Y2
        data_set['one_hot_dictionary'] = one_hot_dictionary
        data_set['template_choices'] = template_choices
        data_set['question_template_list'] = question_template_list
        data_set['answer_template_list'] = answer_template_list
        self.data_set = data_set

        (b, self.m, self.n) = self.data_set['X'].shape

        self.m1 = m1

        self.n1 = n1

        self.m2 = m2

        self.n2 = n2
        
        self.test_on_data_set(0)

    # End method generate_network

    def test_on_data_set(self, data_set_index):

        fp = open('%s/data_set_%d_X1.npy' % (self.DATA_PATH,data_set_index,), 'rb')
        X = np.load(fp)
        fp.close()

        fp = open('%s/data_set_%d_Y1.npy' % (self.DATA_PATH,data_set_index,), 'rb')
        Y1 = np.load(fp)
        fp.close()

        fp = open('%s/data_set_%d_Y2.npy' % (self.DATA_PATH,data_set_index,), 'rb')
        Y2 = np.load(fp)
        fp.close()

        fp = open('%s/data_set_%d_one_hot_dictionary.pck' % (self.DATA_PATH,data_set_index,), 'rb')
        one_hot_dictionary = pickle.load(fp)
        fp.close()

        fp = open('%s/data_set_%d_one_hot_dictionary.pck' % (self.DATA_PATH,data_set_index,), 'rb')
        one_hot_dictionary = pickle.load(fp)
        fp.close()

        fp = open('%s/data_set_%d_template_choices.pck' % (self.DATA_PATH,data_set_index,), 'rb')
        template_choices = pickle.load(fp)
        fp.close()

        fp = open('%s/data_set_%d_question_template_list.pck' % (self.DATA_PATH,data_set_index,), 'rb')
        question_template_list = pickle.load(fp)
        fp.close()

        fp = open('%s/data_set_%d_answer_template_list.pck' % (self.DATA_PATH,data_set_index,), 'rb')
        answer_template_list = pickle.load(fp)
        fp.close()

        data_set = {}
        data_set['X'] = X
        data_set['Y1'] = Y1
        data_set['Y2'] = Y2
        data_set['one_hot_dictionary'] = one_hot_dictionary
        data_set['template_choices'] = template_choices
        data_set['question_template_list'] = question_template_list
        data_set['answer_template_list'] = answer_template_list

        
        # Run regular evaluation too.
        
        evaluation_result = self.model.evaluate(X, [Y1, Y2])

        return evaluation_result
    # End method test_on_data_set
    
    def query_my_knowledge(self, item_to_query):

        X = encode_string(item_to_query, self.data_set['one_hot_dictionary'])
        (m, n) = X.shape
        encoding_of_none = encode_string(None, self.data_set['one_hot_dictionary'])[0]
        X_pad = np.ones((1, self.m, self.n)) * encoding_of_none
        Y1_pad = np.ones((1, self.m, self.n)) * encoding_of_none
        Y2_pad = np.ones((1, self.m, self.n)) * encoding_of_none
        X_pad[0, :m, :] = X

        x_check = np.sum(X_pad, 2)
        y1_check = np.sum(Y1_pad, 2)
        y2_check = np.sum(Y2_pad, 2)
        [Y1_hat,Y2_hat] = self.model.predict(X_pad)

        original_question = inverse_one_hot_dictionary(self.data_set['one_hot_dictionary'],
                                                       X_pad[0, :, :])

        network_answer_array1 = inverse_one_hot_dictionary(self.data_set['one_hot_dictionary'],
                                                    Y1_hat[0, :, :])

        network_answer_array2 = inverse_one_hot_dictionary(self.data_set['one_hot_dictionary'],
                                                    Y2_hat[0, :, :])

        network_answer_string1 = array_to_string(network_answer_array1)

        network_answer_string2 = array_to_string(network_answer_array2)

        return_dict = {
            'original_question' : original_question,

            'network_answer_array1': network_answer_array1,

            'network_answer_string1': network_answer_string1,

            'network_answer_array2' : network_answer_array2,

            'network_answer_string2' : network_answer_string2,
            'X' : X_pad,
            'Y1' : Y1_hat,
            'Y2' : Y2_hat
        }

        return return_dict

    # End function query_my_knowledge

    def add_knowledge(self, knowledge_sentence):

        knowledge_sentence = knowledge_sentence.rstrip().lstrip()

        self.entity_id_and_knowledge['sentence_list'].append(knowledge_sentence)

    # End method add_knowledge

    # Ask question method combines knowledge base with the question and then asks.

    def ask_question(self, question):

        combined_list = copy.deepcopy(self.entity_id_and_knowledge['sentence_list'])

        combined_list.append(question)

        data_string = " . ".join(combined_list)

        return_dict = self.query_my_knowledge(data_string)

        regex_unknown = re.compile('unknown')

        return return_dict

    # End ask_question

    def ask_question_remember_answer(self, question):

        return_dict = self.ask_question(question)

        regex_unknown = re.compile("unknown")

        if question != "help" and regex_unknown.search(return_dict['network_answer_string2']) is None:

            self.add_knowledge(return_dict['network_answer_string2'])

        return return_dict

    # End method ask_question_remember_answer

# End class declaration NN_Entity_1



Using TensorFlow backend.
/media/ryan/venv/alpha/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/media/ryan/venv/alpha/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/media/ryan/venv/alpha/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/media/ryan/venv/alpha/lib/python3.6/site-packages/tenso

In [2]:

# Line below is for building new model!

new_network_flag = True

if new_network_flag:

    test_entity = NN_Entity_1(id_number=1, gen_data_sets=True,
                              max_data_sets=256,
                              start_training_epoch=0,
                              max_training_epoch=2049)
else:
    # Lines below are for using a pre-built model!

    test_entity = NN_Entity_1(id_number=1,
                          nn_file="%s/trained_model_prop_new_2048.h5" % (DATA_PATH,),
                          data_set_file='%s/data_set_0' % (DATA_PATH,))


test_entity_2 = NN_Entity_1(id_number=2,
                          nn_file="%s/trained_model_prop_new_2048.h5" % (DATA_PATH,),
                          data_set_file='%s/data_set_0' % (DATA_PATH,))

# test_entity.add_knowledge("if a2 then a3")
# test_entity.add_knowledge("a1 is false")
# test_entity_2.add_knowledge("if a3 then a1")

test_entity.add_knowledge("a7 is false")
test_entity_2.add_knowledge("if a1 then a9")

# Ask first entity for value of a3.  Get its answer
# and convert answer into format entity 2 can use.

the_question = "what is a1 ?"
return_dict = test_entity.ask_question_remember_answer(the_question)
return_string = return_dict['network_answer_string2']
print(return_string)

return_sentences = return_string.split(".")

return_list = []

regex_help = re.compile("help")

help_flag = False # Only goes true if first entity asks for help.
for sentence in return_sentences:
    new_sentence = sentence.strip()
    if regex_help.search(new_sentence) is None:
        return_list.append(sentence)
        test_entity_2.add_knowledge(sentence)
    else:
        help_flag = True

# If first entity asked for help, then second entity responds with
# dump of its own knowledge.

if help_flag:
    return_dict_2 = test_entity_2.ask_question_remember_answer("help")

    return_string_2 = return_dict_2['network_answer_string2']

    print (return_string_2)

    return_list_2 = return_string_2.split(".")

    for sentence in return_list_2:

        new_sentence = sentence.strip()
        test_entity.add_knowledge(new_sentence)

    # RE-run

    new_dictionary = test_entity.ask_question_remember_answer(the_question)

    new_answer = new_dictionary['network_answer_string2']
else:
    new_answer = None

print (new_answer)

evaluation_result = test_entity.test_on_data_set(0)

print (evaluation_result)



W0924 22:21:27.510431 140600702248768 deprecation_wrapper.py:119] From /media/ryan/venv/alpha/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0924 22:21:31.213778 140600702248768 deprecation_wrapper.py:119] From /media/ryan/venv/alpha/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0924 22:21:31.249529 140600702248768 deprecation_wrapper.py:119] From /media/ryan/venv/alpha/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0924 22:21:31.898897 140600702248768 deprecation_wrapper.py:119] From /media/ryan/venv/alpha/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0924 22

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 65)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, None, 256)    329728      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, None, 256)    525312      lstm_1[0][0]                     
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, None, 65)     16705       lstm_1[0][0]                     
__________________________________________________________________________________________________
dense_2 (D

W0924 22:21:45.597606 140600702248768 deprecation.py:323] From /media/ryan/venv/alpha/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 26s 3ms/step - loss: 1.7008 - activation_1_loss: 0.9543 - activation_2_loss: 0.7465 - activation_1_acc: 0.8304 - activation_2_acc: 0.8744 - val_loss: 0.9846 - val_activation_1_loss: 0.5661 - val_activation_2_loss: 0.4186 - val_activation_1_acc: 0.8589 - val_activation_2_acc: 0.8968
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 25s 3ms/step - loss: 0.8825 - activation_1_loss: 0.5329 - activation_2_loss: 0.3496 - activation_1_acc: 0.8602 - activation_2_acc: 0.9096 - val_loss: 0.7802 - val_activation_1_loss: 0.4982 - val_activation_2_loss: 0.2820 - val_activation_1_acc: 0.8659 - val_activation_2_acc: 0.9127
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 25s 3ms/step - loss: 0.6721 - activation_1_loss: 0.4288 - activation_2_loss: 0.2433 - activation_1_acc: 0.8809 - activation_2

8000/8000 [==============================] - 25s 3ms/step - loss: 0.2709 - activation_1_loss: 0.1676 - activation_2_loss: 0.1033 - activation_1_acc: 0.9388 - activation_2_acc: 0.9617 - val_loss: 0.2615 - val_activation_1_loss: 0.1624 - val_activation_2_loss: 0.0991 - val_activation_1_acc: 0.9408 - val_activation_2_acc: 0.9632
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 25s 3ms/step - loss: 0.2546 - activation_1_loss: 0.1587 - activation_2_loss: 0.0959 - activation_1_acc: 0.9423 - activation_2_acc: 0.9647 - val_loss: 0.2476 - val_activation_1_loss: 0.1536 - val_activation_2_loss: 0.0939 - val_activation_1_acc: 0.9445 - val_activation_2_acc: 0.9648
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 25s 3ms/step - loss: 0.2424 - activation_1_loss: 0.1508 - activation_2_loss: 0.0916 - activation_1_acc: 0.9448 - activation_2_acc: 0.9659 - val_loss: 0.2405 - val_activation_1_loss: 0

...

8000/8000 [==============================] - 26s 3ms/step - loss: 5.6232e-06 - activation_1_loss: 1.2204e-06 - activation_2_loss: 4.4028e-06 - activation_1_acc: 1.0000 - activation_2_acc: 1.0000 - val_loss: 2.9033e-06 - val_activation_1_loss: 1.2164e-06 - val_activation_2_loss: 1.6869e-06 - val_activation_1_acc: 1.0000 - val_activation_2_acc: 1.0000
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 26s 3ms/step - loss: 4.6950e-05 - activation_1_loss: 1.0610e-06 - activation_2_loss: 4.5889e-05 - activation_1_acc: 1.0000 - activation_2_acc: 1.0000 - val_loss: 3.4475e-05 - val_activation_1_loss: 2.0256e-06 - val_activation_2_loss: 3.2450e-05 - val_activation_1_acc: 1.0000 - val_activation_2_acc: 1.0000
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 26s 3ms/step - loss: 3.2837e-05 - activation_1_loss: 2.1513e-06 - activation_2_loss: 3.0686e-05 - activation_1_acc: 1.0000 - activation

8000/8000 [==============================] - 26s 3ms/step - loss: 6.4341e-05 - activation_1_loss: 3.6518e-06 - activation_2_loss: 6.0690e-05 - activation_1_acc: 1.0000 - activation_2_acc: 1.0000 - val_loss: 2.9714e-04 - val_activation_1_loss: 2.8728e-05 - val_activation_2_loss: 2.6841e-04 - val_activation_1_acc: 1.0000 - val_activation_2_acc: 0.9999
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 26s 3ms/step - loss: 1.9276e-04 - activation_1_loss: 1.6088e-05 - activation_2_loss: 1.7668e-04 - activation_1_acc: 1.0000 - activation_2_acc: 1.0000 - val_loss: 4.2335e-05 - val_activation_1_loss: 1.8483e-06 - val_activation_2_loss: 4.0487e-05 - val_activation_1_acc: 1.0000 - val_activation_2_acc: 1.0000
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 26s 3ms/step - loss: 1.3467e-04 - activation_1_loss: 3.5757e-05 - activation_2_loss: 9.8917e-05 - activation_1_acc: 1.0000 - activation

8000/8000 [==============================] - 26s 3ms/step - loss: 6.1254e-05 - activation_1_loss: 3.7072e-06 - activation_2_loss: 5.7546e-05 - activation_1_acc: 1.0000 - activation_2_acc: 1.0000 - val_loss: 4.9119e-05 - val_activation_1_loss: 1.3791e-05 - val_activation_2_loss: 3.5328e-05 - val_activation_1_acc: 1.0000 - val_activation_2_acc: 1.0000
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 26s 3ms/step - loss: 9.3794e-05 - activation_1_loss: 6.5693e-05 - activation_2_loss: 2.8101e-05 - activation_1_acc: 1.0000 - activation_2_acc: 1.0000 - val_loss: 1.7873e-05 - val_activation_1_loss: 4.7325e-06 - val_activation_2_loss: 1.3141e-05 - val_activation_1_acc: 1.0000 - val_activation_2_acc: 1.0000
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 26s 3ms/step - loss: 2.9973e-04 - activation_1_loss: 1.2078e-04 - activation_2_loss: 1.7895e-04 - activation_1_acc: 1.0000 - activation

8000/8000 [==============================] - 26s 3ms/step - loss: 1.3750e-04 - activation_1_loss: 6.7448e-05 - activation_2_loss: 7.0054e-05 - activation_1_acc: 1.0000 - activation_2_acc: 1.0000 - val_loss: 3.0884e-04 - val_activation_1_loss: 2.0914e-05 - val_activation_2_loss: 2.8792e-04 - val_activation_1_acc: 1.0000 - val_activation_2_acc: 0.9999
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 26s 3ms/step - loss: 1.0077e-04 - activation_1_loss: 4.2484e-06 - activation_2_loss: 9.6522e-05 - activation_1_acc: 1.0000 - activation_2_acc: 1.0000 - val_loss: 3.7979e-05 - val_activation_1_loss: 1.1630e-05 - val_activation_2_loss: 2.6350e-05 - val_activation_1_acc: 1.0000 - val_activation_2_acc: 1.0000
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 26s 3ms/step - loss: 2.7872e-05 - activation_1_loss: 1.1963e-05 - activation_2_loss: 1.5909e-05 - activation_1_acc: 1.0000 - activation

8000/8000 [==============================] - 26s 3ms/step - loss: 7.8592e-05 - activation_1_loss: 3.8323e-05 - activation_2_loss: 4.0269e-05 - activation_1_acc: 1.0000 - activation_2_acc: 1.0000 - val_loss: 1.0379e-04 - val_activation_1_loss: 6.4643e-05 - val_activation_2_loss: 3.9152e-05 - val_activation_1_acc: 1.0000 - val_activation_2_acc: 1.0000
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 26s 3ms/step - loss: 6.1062e-05 - activation_1_loss: 1.9089e-05 - activation_2_loss: 4.1973e-05 - activation_1_acc: 1.0000 - activation_2_acc: 1.0000 - val_loss: 9.1353e-06 - val_activation_1_loss: 2.7614e-06 - val_activation_2_loss: 6.3738e-06 - val_activation_1_acc: 1.0000 - val_activation_2_acc: 1.0000
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 26s 3ms/step - loss: 3.8219e-05 - activation_1_loss: 5.2368e-06 - activation_2_loss: 3.2982e-05 - activation_1_acc: 1.0000 - activation